In [ ]:
# install for connection to postgres database (local)
pip install psycopg2

import csv
import psycopg2
import pandas as pd

# install for connection to the database using DB2 (cloud)
#!pip install sqlalchemy==1.3.9
#!pip install ibm_db_sa
#!pip install ipython-sql
print('Project libraries has been successfully installed!')

In [ ]:
%load_ext sql

In [ ]:
%sql ibm_db_sa://

In [ ]:
# create connection to postgreSQL database
conn = psycopg2.connect(
    host = 'localhost',
    database = 'postgres', 
    user = 'postgres', 
    password = 'chuksoo',  
    port = '5432')
print('Connection to database is successfully')

In [ ]:
# function to read from database
def read(conn, read_):
    print('Read')
    cursor = conn.cursor()
    cursor.execute(read_)
    for row in cursor:
        print(f'row = {row}')
    print()
    
# function to create in postgre database     
def create(conn, create_):
    cursor = conn.cursor() # create cursor object
    cursor.execute(create_) # execute query
    conn.commit() # commit query to database
    print('Table have been created successfull!!!')
    #read(conn)
    
# function to insert in postgre database     
def insert(conn, insert_):
    cursor = conn.cursor()
    cursor.execute(insert_)
    conn.commit()
    print('Records have been successfully inserted!!!')
    #read(conn)
    
# function to update table
def update(conn, update_):
    print('Update')
    cursor = conn.cursor()
    cursor.execute(update_)
    conn.commit()
    #read(conn)
    
# function to delete in postgre database
def delete(conn, delete_):
    print('Delete')
    cursor = conn.cursor()
    cursor.execute(delete_)
    conn.commit()
    #read(conn)

# close the cursor and connection to the server 
def close():
    cursor.close()
    conn.close()   
    
# function to create pandas dataframe
def create_pandas_df(sql_query, database=conn):
    table = pd.read_sql_query(sql_query, database)
    return table

In [ ]:
# create table SpaceX
create_ = '''
            DROP TABLE IF EXISTS SpaceX;
            CREATE TABLE SpaceX
                (
                    Date DATE NULL,
                    Time TIME NULL,
                    BoosterVersion VARCHAR(50) NULL,
                    LaunchSite VARCHAR(50) NULL,
                    Payload VARCHAR(100) NULL,
                    PayloadMassKG INT NULL,
                    Orbit VARCHAR(50) NULL,
                    Customer VARCHAR(100) NULL,
                    MissionOutcome VARCHAR(50) NULL,
                    LandingOutcome VARCHAR(100) NULL
                );
            '''
create(conn, create_)

In [ ]:
# loading the csv file
cursor = conn.cursor()
with open('Spacex.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader) # Skip the header row.
    for row in reader:
        cursor.execute(
        "INSERT INTO SpaceX VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
        row
    )
conn.commit()
print('CSV file inserted into database successfully!!!')

In [ ]:
# read to see that data has been uploaded in database
read_ = '''
        SELECT *
        FROM   SpaceX
        LIMIT 10
        '''
read(conn, read_)

In [ ]:
# output postgre query in pandas dataframe
spacex = create_pandas_df(read_, database=conn)
spacex.head(10)

In [ ]:
task_1 = '''
        SELECT DISTINCT LaunchSite 
        FROM SpaceX
'''
create_pandas_df(task_1, database=conn)

In [ ]:
task_2 = '''
        SELECT *
        FROM SpaceX
        WHERE LaunchSite LIKE 'CCA%'
        LIMIT 5
        '''
create_pandas_df(task_2, database=conn)

In [ ]:
task_3 = '''
        SELECT SUM(PayloadMassKG) AS Total_PayloadMass
        FROM SpaceX
        WHERE Customer LIKE 'NASA (CRS)'
        '''
create_pandas_df(task_3, database=conn)

In [ ]:
task_4 = '''
        SELECT AVG(PayloadMassKG) AS Avg_PayloadMass
        FROM SpaceX
        WHERE BoosterVersion = 'F9 v1.1'
        '''
create_pandas_df(task_4, database=conn)

In [ ]:
task_5 = '''
        SELECT MIN(Date) AS FirstSuccessfull_landing_date
        FROM SpaceX
        WHERE LandingOutcome LIKE 'Success (ground pad)'
        '''
create_pandas_df(task_5, database=conn)

In [ ]:
task_6 = '''
        SELECT BoosterVersion
        FROM SpaceX
        WHERE LandingOutcome = 'Success (drone ship)'
            AND PayloadMassKG > 4000 
            AND PayloadMassKG < 6000
        '''
create_pandas_df(task_6, database=conn)

In [ ]:
task_7a = '''
        SELECT COUNT(MissionOutcome) AS SuccessOutcome
        FROM SpaceX
        WHERE MissionOutcome LIKE 'Success%'
        '''

task_7b = '''
        SELECT COUNT(MissionOutcome) AS FailureOutcome
        FROM SpaceX
        WHERE MissionOutcome LIKE 'Failure%'
        '''
print('The total number of successful mission outcome is:')
display(create_pandas_df(task_7a, database=conn))
print()
print('The total number of failed mission outcome is:')
create_pandas_df(task_7b, database=conn)

In [ ]:
task_8 = '''
        SELECT BoosterVersion, PayloadMassKG
        FROM SpaceX
        WHERE PayloadMassKG = (
                                SELECT MAX(PayloadMassKG)
                                FROM SpaceX
                                )
        ORDER BY BoosterVersion
        '''
create_pandas_df(task_8, database=conn)

In [ ]:
task_9 = '''
        SELECT BoosterVersion, LaunchSite, LandingOutcome
        FROM SpaceX
        WHERE LandingOutcome LIKE 'Failure (drone ship)'
            AND Date BETWEEN '2015-01-01' AND '2015-12-31'
        '''
create_pandas_df(task_9, database=conn)

In [ ]:
task_10 = '''
        SELECT LandingOutcome, COUNT(LandingOutcome)
        FROM SpaceX
        WHERE DATE BETWEEN '2010-06-04' AND '2017-03-20'
        GROUP BY LandingOutcome
        ORDER BY COUNT(LandingOutcome) DESC
        '''
create_pandas_df(task_10, database=conn)